<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/07Jul2022_MLB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/mlb_batting.zip -O mlb.zip

In [8]:
import shutil
from pathlib import Path
import pandas as pd

In [3]:
shutil.unpack_archive('mlb.zip', extract_dir='mlb')

In [46]:
files=list(Path('mlb').glob('**/*.csv'))
keys=[afile.as_posix().split('/')[1:] for afile in files]
keys=[tuple((key[0], key[1], key[-1].split('.')[0])) for key in keys]

In [141]:
df_list=[]
days=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for afile in files:
    df=pd.read_csv(afile).iloc[:,1:]
    df['home']=df['OPP'].str.startswith('vs').astype(int)
    df['win']=(df['Result'].str[0]=='W').astype(int)
    df=df.loc[df['Date'].str[:3].isin(days),['Date','AB','HR','home','win']]
    df[['day','Date']]=df.loc[:,'Date'].str.split(' ',expand=True)
    df_list.append(df)

In [145]:
df=pd.concat(df_list,keys=keys)
df.index=df.index.droplevel(3)
df.index.names=['team','player','year']
df=df.reset_index().set_index(['team','player'])
df['date']=pd.to_datetime(df['Date'] + '/' + df['year'])
df=df.drop(['year','Date'],axis=1)

In [147]:
df

AB HR  home  win  day       date
team                player                                          
los-angeles-dodgers Freddie Freeman  4  0     1    0  Wed 2019-10-09
                    Freddie Freeman  5  0     0    0  Mon 2019-10-07
                    Freddie Freeman  4  0     0    1  Sun 2019-10-06
                    Freddie Freeman  3  0     1    1  Fri 2019-10-04
                    Freddie Freeman  4  1     1    0  Thu 2019-10-03
...                                 .. ..   ...  ...  ...        ...
pittsburgh-pirates  Oneil Cruz       5  0     1    0  Wed 2022-06-22
                    Oneil Cruz       4  0     1    1  Tue 2022-06-21
                    Oneil Cruz       5  0     1    1  Mon 2022-06-20
                    Oneil Cruz       4  1     1    0  Sun 2021-10-03
                    Oneil Cruz       5  0     1    1  Sat 2021-10-02

[252919 rows x 6 columns]

In [148]:
df[['AB','HR']]=df[['AB','HR']].astype(int)

In [149]:
df

AB  HR  home  win  day       date
team                player                                            
los-angeles-dodgers Freddie Freeman   4   0     1    0  Wed 2019-10-09
                    Freddie Freeman   5   0     0    0  Mon 2019-10-07
                    Freddie Freeman   4   0     0    1  Sun 2019-10-06
                    Freddie Freeman   3   0     1    1  Fri 2019-10-04
                    Freddie Freeman   4   1     1    0  Thu 2019-10-03
...                                  ..  ..   ...  ...  ...        ...
pittsburgh-pirates  Oneil Cruz        5   0     1    0  Wed 2022-06-22
                    Oneil Cruz        4   0     1    1  Tue 2022-06-21
                    Oneil Cruz        5   0     1    1  Mon 2022-06-20
                    Oneil Cruz        4   1     1    0  Sun 2021-10-03
                    Oneil Cruz        5   0     1    1  Sat 2021-10-02

[252919 rows x 6 columns]

In [160]:
df.pivot_table(index=['team'],values=['AB','HR'],columns='home',aggfunc='sum')

AB          HR      
home                       0      1    0     1
team                                          
arizona-diamondbacks    8471   8392  292   255
atlanta-braves         15928  15415  688   675
baltimore-orioles       9261   8947  325   373
boston-red-sox         17587  16887  631   692
chicago-cubs           13552  13218  420   428
chicago-white-sox      14526  13922  478   547
cincinnati-reds        14923  13884  528   548
cleveland-guardians     8407   7804  290   253
colorado-rockies       12585  12451  416   532
detroit-tigers         15646  15138  612   587
houston-astros         15932  15359  514   556
kansas-city-royals     13010  12810  478   403
los-angeles-angels     15179  14993  504   548
los-angeles-dodgers    19256  18268  738   806
miami-marlins          10503  10079  367   313
milwaukee-brewers      17023  15945  597   553
minnesota-twins        10581  10041  474   451
new-york-mets          20478  19081  697   662
new-york-yankees       21731  21387  960  1020
oakland-athletics      13290  12652  398   298
philadelphia-phillies  17268  16805  655   689
pittsburgh-pirates      7615   7266  235   213
san-diego-padres       19326  17936  683   739
san-francisco-giants   17192  16390  704   615
seattle-mariners       16740  15871  654   713
st-louis-cardinals     20169  19287  933   822
tampa-bay-rays          8549   8611  312   309
texas-rangers          11782  11281  432   457
toronto-blue-jays       9310   9055  418   430
washington-nationals   17379  16551  553   542